In [1]:
#  pip install yargy
#  pip install pandas

import pandas as pd

from yargy.tokenizer import MorphTokenizer
from yargy import rule, Parser, or_, and_, not_
from yargy.predicates import eq, type as type_, in_, normalized, gram, is_capitalized, dictionary
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact


# Hi there!   In this notebook we will extract data with rule-base yargy parser



Yargy is an Earley parser similar to Tomita parser. Yargy uses rules and dictionaries to extract structured information from Russian texts. 
Find more at https://github.com/natasha/yargy

In [2]:
df = pd.read_csv(r'test_data.csv')


First let's briefly check our data, it's shape, integrity and type

In [3]:
df.shape


(480, 4)

In [4]:
df.dtypes


dlg_id     int64
line_n     int64
role      object
text      object
dtype: object

In [5]:
assert df.isna().sum().sum() == 0


In [6]:
df.duplicated().sum()


0

So far so good 

# Now we want to draw a conversation matrix to understand what exactly is the data we a looking for.

In [7]:
text = [str(k) + "|" + str(a) + "|" + str(b) + "|" + str(c) +"|" + str(d) for k, (a, b, c, d) in enumerate(zip(df.dlg_id, df.line_n, df.role, df.text))]

In [8]:
#  This is what our task is about: 
#
#  Главные задачи, которые должен выполнять скрипт:
#  a) Извлекать реплики с приветствием – где менеджер поздоровался. 
#  b) Извлекать реплики, где менеджер представил себя. 
#  c) Извлекать имя менеджера. 
#  d) Извлекать название компании. 
#  e) Извлекать реплики, где менеджер попрощался.
#  f) Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»


Note: It's not clear how to treat phrases like "248|2|84|manager|Все хорошо", '338|5|1|manager|Да это анастасия'.
As they are way too unformal. I suppose we are expected to collect formal ones.
So for the sake of this task we decide that "338|5|1|manager|Да это анастасия" is proper way to say hi and skip others.

In [9]:
#  Let's explore what manager lines look like, I'm not sure if we can publish them due to task restriction.
#  So I'll leave here first few sentences as an example. 

corpus = ['1|0|1|manager|Алло здравствуйте',
'3|0|3|manager|Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается',
'108|0|108|manager|Всего хорошего до свидания']


# We split data by role and look through it to hopefully get insights. 

In [10]:
pd.set_option('display.max_rows', None)
pd.options.display.max_colwidth = 900
df[df.role=='manager']


,dlg_id,line_n,role,text
1,0,1,manager,Алло здравствуйте
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
5,0,5,manager,Угу ну возможно вы рассмотрите и другие варианты видите это хорошая практика сравнивать
8,0,8,manager,Угу а на что вы обращаете внимание при выборе
11,0,11,manager,Что для вас приоритет
15,0,15,manager,Ну у вас срок заканчивается поэтому мы набрали + ко всему
29,0,29,manager,А так нет не только поэтому просто я обратила внимание и вы пользуетесь виджетами мы там с вами сотрудничали
34,0,34,manager,А если вы 19 являетесь то лучше то идти бесплатно
36,0,36,manager,Ага хорошо
45,0,45,manager,Индивидуальным поэтому не все то есть сотрудничают с кем то сейчас да кому то например не с кем сейчас справляться кто то сравнивал


In [11]:
df[df.role=='client']


,dlg_id,line_n,role,text
0,0,0,client,Алло
2,0,2,client,Добрый день
4,0,4,client,Ага
6,0,6,client,Да мы работаем с компанией которая нам подливает поэтому спасибо огромное
7,0,7,client,Как как бы уже до этого момента работаем все устраивает + у нас сопровождение поэтому
9,0,9,client,Как бы нет
10,0,10,client,Да на выбор я уже не обращаю внимание я уже дома выбрал
12,0,12,client,Поэтому типа много параметров
13,0,13,client,Выбор не выбор есть история сотрудничества которая более важна потому что вот смотрите вы же мне 1 раз звоните
14,0,14,client,А работаем мы там а уже давно а почему звоните только сейчас а не в течении своего времени когда работают


Now we have enough information to draw conversation matrix. 

# As we developed intuition how our data should look like, we start write rules for our parser. 

We take to account not only phrases we find in our data set, but also phrases we could find in other possible business conversations.
To give our script more generalization power 

As attributes we use word's part of speech and grammemes.

In [36]:
#  Rules to find lines where manager says 'Hi'

Greet = fact (
    'Greet', ['greet']
)

greet = rule(or_(eq('добрый'), eq('добрый'), eq('доброе')), or_(eq('день'),eq('утро'),eq('вечер')))
greet1 = rule(or_(eq('здраствуйте'), eq('привет'), eq('здрасьте'), eq('здраствуй'), eq('здравствуйте'), eq('здравствуй')))
greet = or_(greet, greet1).interpretation(Greet.greet).interpretation(Greet)
greet  = Parser(greet)

#  This includes alot of combinations to say Hi. 
#  We add few misspelled words. 

#  This doesn't include informal greetings like "Приветики" etc.


In [37]:
#  Rules to find lines where manager inroduces himself

Manager = fact(
    "Manager", ["name"]
)

Intro = fact(
    "Intro", ["phrase"]
)

eto = and_(gram('NPRO'), gram('neut'), gram('nomn'), gram('sing'))
pers_self = rule(and_(gram('1per'), gram('NPRO'), gram('sing')))
pers_call = and_(gram('3per'), gram('VERB'), gram('tran')) 
pers_surname = rule(gram('Surn').repeatable().optional())
pers_first_name = rule(and_(gram('Name'),not_(gram('Abbr'))))
pers_name = rule(pers_first_name, pers_surname).interpretation(Manager.name)
pers_name1 = rule(pers_first_name, pers_surname)
manager = rule(or_(rule(pers_self, pers_call, pers_name), rule(pers_self, pers_name,  pers_call), rule(eto, pers_name)))
manager_intro = rule(or_(rule(pers_self, pers_call, pers_name1), rule(pers_self, pers_name1,  pers_call), rule(eto, pers_name1))).interpretation(Intro.phrase)
manager_intro = manager_intro.interpretation(Intro)
manager = manager.interpretation(Manager)

manager = Parser(manager)
manager_intro = Parser(manager_intro)


# We made sure that name and second name (if manager says one) should have Name, Surname attributes.


In [14]:
#  Rules to find lines where company name is mentioned 

Company = fact('Company', ['c_name'])

org = gram('Orgn')
engl = type_('LATIN')
exclude = rule(and_(gram('ADJF'), not_(gram('plur')), not_(gram('Apro'))))
comp = rule(or_(eq('компания'), eq('компании'), eq('компанию')))
company = rule(comp, or_(and_(gram('NOUN'), not_(gram('PREP')), not_(gram('CONJ'))).repeatable(), exclude), eq('бизнес').optional())
company1 = rule(comp, org.repeatable())
company2 = rule(comp, engl.repeatable())
company = rule(or_(company, company1, company2)).interpretation(Company.c_name)
company = company.interpretation(Company)
company = Parser(company)


In [15]:
#  Rules to find lines where manager says 'Good bye' 

Fare = fact (
    'Fare', ['fare']
)

farewell = rule(eq('всего'), or_(eq('доброго'), eq('хорошего')))
farewell1 = rule(or_(eq('удачного'), eq('хорошего')),  or_(eq('дня'), eq('вечера')))
farewell2 = rule(eq('доброй'), eq('ночи'))
farewell3 = rule(eq('досвидания'))
farewell4 = rule(eq('до'), eq('свидания'))
farewell = or_(farewell, farewell1, farewell2, farewell3, farewell4).interpretation(Fare.fare).interpretation(Fare)
farewell = Parser(farewell)


In [38]:
#  Quick note:  I don't use functions here and further to make code look streamlined and easier to read. 

intro = []
result = []

df['name'] = False
df['greet'] = False
df['farewell'] = False
df['has_greet'] = False
df['is_polite'] = False
df['introduction'] = False
df['has_farewell'] = False
df['company_label'] = False
df['has_introduction'] = False
df['has_company_label'] = False

#  We loop through manager lines and look at each line to find our data of interest.

for line in df[df.role=='manager'].iterrows():
  
    greet_ = list(greet.findall(line[1].text.lower()))
    manager_ = list(manager.findall(line[1].text.lower()))
    manager_intro_ = list(manager_intro.findall(line[1].text.lower()))
    company_ = list(company.findall(line[1].text.lower()))
    farewell_ = list(farewell.findall(line[1].text.lower()))

    if greet_:   
        df['has_greet'].iloc[line[0]] = True
        df['greet'].iloc[line[0]] = greet_[0].fact.greet
           
    if manager_:  
        if manager_intro_:
            response = manager_intro_[0].fact.phrase
            intro.append(response)      
        df['has_introduction'].iloc[line[0]] = True
        df['introduction'].iloc[line[0]] = response
        df['name'].iloc[line[0]] = manager_[0].fact.name
    
    if company_:  
        df['has_company_label'].iloc[line[0]] = True
        df['company_label'].iloc[line[0]] = company_[0].fact.c_name
    
    if farewell_:     
        df['has_farewell'].iloc[line[0]] = True
        df['farewell'].iloc[line[0]] = farewell_[0].fact.fare


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [39]:
#  Up to the moment we collected all data we need 
#  Time to find which managers were polite 

details = []
polite_list = []

for i in df.dlg_id.unique():   
    name_tag = df[(df.dlg_id==i) & (df.name!=False)].name.to_list()
    if not name_tag:
        name_tag = ['Unknown']
    company_tag = df[(df.dlg_id==i) & (df.has_company_label)].company_label.to_list()
    if not company_tag:
        company_tag = ['UNK']
    if (True in df[(df.dlg_id==i) & df.has_greet].has_greet.to_list()) & (True in df[(df.dlg_id==i) & df.has_farewell].has_farewell.to_list()):
        is_polite_str = ""
        df.is_polite[(df.dlg_id==i) & df.has_greet] = True
        df.is_polite[(df.dlg_id==i) & df.has_farewell] = True
        if name_tag:
            polite_list.append(name_tag[0])
    else:
        is_polite = 'NOT'
    sentence = f'Call #{i} manager {name_tag[0].capitalize()} from {company_tag[0]} is {is_polite_str} polite'
    details.append(sentence)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
#  We can use df[df.is_polite] command to see greeting and farewell lines of polite managers.

df[df.is_polite].head() 


,dlg_id,line_n,role,text,name,greet,farewell,has_greet,is_polite,introduction,has_farewell,company_label,has_introduction,has_company_label
1,0,1,manager,Алло здравствуйте,False,здравствуйте,False,True,True,False,False,False,False,False
108,0,108,manager,Всего хорошего до свидания,False,False,всего хорошего,False,True,False,True,False,False,False
110,1,1,manager,Алло здравствуйте,False,здравствуйте,False,True,True,False,False,False,False,False
162,1,53,manager,Угу да вижу я эту почту хорошо тогда исправлю на эту будем ждать ответа всего хорошего,False,False,всего хорошего,False,True,False,True,False,False,False
163,1,54,manager,До свидания,False,False,до свидания,False,True,False,True,False,False,False


# Now we can give answers the task asks:

a) Извлекать реплики с приветствием – где менеджер поздоровался.

In [21]:
df[df.has_greet].greet.to_list()


['здравствуйте', 'здравствуйте', 'здравствуйте', 'добрый день']

In [22]:
#  'здравствуйте', 'здравствуйте', 'здравствуйте', 'добрый день'


b) Извлекать реплики, где менеджер представил себя.

In [23]:
#  Full line where manager introduces herself

df[df.has_introduction].text.to_list()


['Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается',
 'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается',
 'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там',
 'Добрый меня максим зовут компания китобизнес удобно говорить',
 'Да это анастасия']

In [24]:
#  'Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается',
#  'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается',
#  'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там',
#  'Добрый меня максим зовут компания китобизнес удобно говорить'


In [25]:
#  Exact introduction words

intro


['меня зовут ангелина',
 'меня зовут ангелина',
 'меня зовут ангелина',
 'меня максим зовут',
 'это анастасия']

In [26]:
#  'меня зовут ангелина',
#  'меня зовут ангелина',
#  'меня зовут ангелина',
#  'меня максим зовут',
#  'это анастасия'


c) Извлекать имя менеджера. 

In [27]:
df[df.name!=False].name.to_list()


['ангелина', 'ангелина', 'ангелина', 'максим', 'анастасия']

In [28]:
#  'ангелина', 'ангелина', 'ангелина', 'максим', 'анастасия'


d) Извлекать название компании. 

In [29]:
df[df.company_label!=False].company_label.to_list()


['компания диджитал бизнес',
 'компания диджитал бизнес',
 'компания диджитал бизнес',
 'компания китобизнес']

In [30]:
#  'компания диджитал бизнес',
#  'компания диджитал бизнес',
#  'компания диджитал бизнес',
#  'компания китобизнес'


e) Извлекать реплики, где менеджер попрощался.

In [31]:
df[df.has_farewell].farewell.to_list()


['всего хорошего',
 'всего хорошего',
 'до свидания',
 'всего доброго',
 'до свидания',
 'до свидания']

In [32]:
#  'Всего хорошего до свидания',
#  'Угу да вижу я эту почту хорошо тогда исправлю на эту будем ждать ответа всего хорошего',
#  'До свидания',
#  'Угу все хорошо да понедельника тогда всего доброго',
#  'Во вторник все ну с вами да тогда до вторника до свидания',
#  'Ну до свидания хорошего вечера'


f) Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

In [33]:
details


['Call #0 manager Ангелина from компания диджитал бизнес is  polite',
 'Call #1 manager Ангелина from компания диджитал бизнес is  polite',
 'Call #2 manager Ангелина from компания диджитал бизнес is  polite',
 'Call #3 manager Максим from компания китобизнес is  polite',
 'Call #4 manager Unknown from UNK is  polite',
 'Call #5 manager Анастасия from UNK is  polite']

In [34]:
#  'Call #0 manager Ангелина from компания диджитал бизнес is  polite',
#  'Call #1 manager Ангелина from компания диджитал бизнес is  polite',
#  'Call #2 manager Ангелина from компания диджитал бизнес is NOT polite',
#  'Call #3 manager Максим from компания китобизнес is  polite',
#  'Call #4 manager Unknown from UNK is NOT polite',
#  'Call #5 manager Анастасия from UNK is NOT polite'


# Now let's compare our results with our matrix we draw in the beginning. 
# Hooray, we found all relevant data. 

For convenience extracted data stores in pandas data frame, so we can look all the data we found with respect to it's dialog id, line number and so on any time

In [40]:
df.head()


,dlg_id,line_n,role,text,name,greet,farewell,has_greet,is_polite,introduction,has_farewell,company_label,has_introduction,has_company_label
0,0,0,client,Алло,False,False,False,False,False,False,False,False,False,False
1,0,1,manager,Алло здравствуйте,False,здравствуйте,False,True,True,False,False,False,False,False
2,0,2,client,Добрый день,False,False,False,False,False,False,False,False,False,False
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,ангелина,False,False,False,False,меня зовут ангелина,False,компания диджитал бизнес,True,True
4,0,4,client,Ага,False,False,False,False,False,False,False,False,False,False


# Pros, Cons and Afterthoughts

The strong sides of Rule-Based extraction are:
    
high execution speed and low computational costs
    
simpleness and cleareness of extracting rules 

If our data is well structured - therefore has low variance, Rule-Based extraction is all we need.

Working process looks like: we write rules, test, evaluate to see what data is not extracted, correct rules, test etc.
When our rules cover all test data, we add more data to look how our current set of rules describes it.
Then we add new rules, test and so on.


    

Obviously this means our algorithm has low generalization ability.

All the time we will keep finding new exceptions.

Is there anything we can do, so we could use less strict rules?

Let's find this out and try CRF statistical model in my next notebook.
# url placeholder


Realy, check this out it's rather interesting